In [ ]:
# Get the template
from app.src.helpers.template_getter import get_template
contract = get_template('meat_sale')

print(contract.to_sym())

In [ ]:
obs = contract.template_strings['obligations']
pows = contract.template_strings['powers']

print('== OBLIGATIONS ==')
for x in obs:
    print(f'- {x}: {obs[x]}\n')

print('== POWERS ==')
for x in pows:
    print(f'- {x}: {pows[x]}\n')

In [ ]:
# Select the parameter
from app.templates.meat_sale.nl_template import parameters

pki_dict = {i+1: pk for i,pk in enumerate(parameters)}

for pki in pki_dict:
    print(f'{pki} - {pki_dict[pki]}')

selected_pki = input('Select a parameter: ')
selected_pk = pki_dict[int(selected_pki)]
selected_parm = parameters[selected_pk][0]

print(f'\nParameter: {selected_pk}')

# The selected parameter will trigger some other things, e.g. which grammar/frameset to use...


In [ ]:
from app.src.operations.helpers.default_event_getter import DefaultEventGetter

# Extract the default event
deg = DefaultEventGetter()

# Handle the case where there is none...
default_event = deg.get(contract.contract_spec, selected_parm.config)

print(f'Default Event: {default_event.to_sym()}')


In [ ]:
#default_event = None

In [ ]:
from app.src.operations.configs import OpCode
from app.src.grammar.grammar_generator import GrammarGenerator
from app.src.grammar.grammar_config import GrammarGeneratorConfig

# Generate the grammar from the contract template
gg = GrammarGenerator()

# These should probably come from the parameter...
op_codes = [OpCode.ADD_TRIGGER, OpCode.REFINE_PREDICATE, OpCode.ADD_NORM] 

grammar_config = GrammarGeneratorConfig(op_codes)

grammar_root = gg.generate(contract, grammar_config)


In [ ]:
from app.src.grammar.grammar_selector import GrammarSelector
from app.src.grammar.manual_node_selector import ManualGrammarNodeSelector

# Run the selection so the user can choose the text
inner_selector = ManualGrammarNodeSelector()
gs = GrammarSelector(inner_selector)
selection = gs.select(grammar_root)

In [ ]:
from app.src.frames.frame_checker_constuctor import FrameCheckerConstructor

frame_checker = FrameCheckerConstructor.construct()
frames = frame_checker.check_all_frames(selection.nodes)

for x in frames:
    print(type(x), x.to_text())

main_frame = frames[0]
op_code = main_frame.op_code

In [ ]:
# Create the basket
from app.classes.grammar.selected_node import Basket
basket = Basket()
basket.default_event = default_event
basket.initial_norm = contract.contract_spec.__dict__[selected_parm.config.norm_type][selected_parm.config.norm_id]


print(basket.initial_norm.to_sym())

In [ ]:
# Generate the Symboleo
symboleo_obj = selection.to_obj(basket)
print(symboleo_obj.to_sym())

In [ ]:
# Updating
from app.src.operations.contract_updater_builder import ContractUpdaterBuilder

contract_updater = ContractUpdaterBuilder.build()

updated_contract = contract_updater.update(op_code, contract, symboleo_obj, selected_parm.config)


In [ ]:
print(updated_contract.to_sym())